In [26]:
import cv2
import numpy as np

def add_transparent_image(background, foreground, x_offset=None, y_offset=None):
    bg_h, bg_w, bg_channels = background.shape
    fg_h, fg_w, fg_channels = foreground.shape

    assert bg_channels == 3, f'background image should have exactly 3 channels (RGB). found:{bg_channels}'
    assert fg_channels == 4, f'foreground image should have exactly 4 channels (RGBA). found:{fg_channels}'

    # center by default
    if x_offset is None: x_offset = (bg_w - fg_w) // 2
    if y_offset is None: y_offset = (bg_h - fg_h) // 2

    w = min(fg_w, bg_w, fg_w + x_offset, bg_w - x_offset)
    h = min(fg_h, bg_h, fg_h + y_offset, bg_h - y_offset)

    if w < 1 or h < 1: return

    # clip foreground and background images to the overlapping regions
    bg_x = max(0, x_offset)
    bg_y = max(0, y_offset)
    fg_x = max(0, x_offset * -1)
    fg_y = max(0, y_offset * -1)
    foreground = foreground[fg_y:fg_y + h, fg_x:fg_x + w]
    background_subsection = background[bg_y:bg_y + h, bg_x:bg_x + w]

    # separate alpha and color channels from the foreground image
    foreground_colors = foreground[:, :, :3]
    alpha_channel = foreground[:, :, 3] / 255  # 0-255 => 0.0-1.0

    # construct an alpha_mask that matches the image shape
    alpha_mask = np.dstack((alpha_channel, alpha_channel, alpha_channel))

    # combine the background with the overlay image weighted by alpha
    composite = background_subsection * (1 - alpha_mask) + foreground_colors * alpha_mask

    # overwrite the section of the background image that has been updated
    background[bg_y:bg_y + h, bg_x:bg_x + w] = composite

    # LABEL
    label = background*0
    label_subsection = label[bg_y:bg_y + h, bg_x:bg_x + w]
    label_composite = label_subsection * (1 - alpha_mask) + foreground_colors * alpha_mask
    label[bg_y:bg_y + h, bg_x:bg_x + w] = label_composite
    label = label[:,:,1] # return only the first channel for labelling

    return label

# PARAMETERS
num_fruit_images = 15           # SET VALUE (# fruit images)
num_bg_images = 1               # SET VALUE (# background images)
num_img_per_fruit = 5           # SET VALUE (# image per fruit)

# OFFSET OF THE FRUIT LOCATION
x_offset_min = -200             # SET VALUE (LEFT)
x_offset_max = 200              # SET VALUE (RIGHT)
y_offset_min = -100             # SET VALUE (UP)
y_offset_max = 50               # SET VALUE (DOWN)

# ROTATION ANGLE
rot_min = 0                     # SET VALUE (MIN ANGLE)
rot_max = 45                    # SET VALUE (MAX ANGLE)

i = 0       # image_numbering
# LABEL - background: 0, redapple: 1, greenapple: 2, orange: 3, mango: 4, capsicum: 5
label_bg = 0
label_fruit = 1

for bg in range(num_bg_images):
    for fruit in range(num_fruit_images):
        # if reaches next instance of fruit
        if (fruit != 0) & (fruit % num_img_per_fruit == 0):
            label_fruit += 1
        
        background = cv2.imread(f'background/calib_{bg}.png')
        overlay = cv2.imread(f'fruits/calib_{fruit}.png', cv2.IMREAD_UNCHANGED)  # IMREAD_UNCHANGED => open image with the alpha channel

        # Random Position
        x_offset = np.random.randint(x_offset_min, x_offset_max)
        y_offset = np.random.randint(y_offset_min, y_offset_max)

        # Random Rotation
        [height,width] = overlay.shape[:2]
        theta = np.random.randint(rot_min, rot_max)
        M = cv2.getRotationMatrix2D((width/2, height/2), theta, 1.0)
        overlay = cv2.warpAffine(overlay, M, (width, height))

        # Label and Save
        img = background.copy()
        mask = add_transparent_image(img, overlay, x_offset, y_offset)
        idx_fruit = np.nonzero(mask)
        mask[idx_fruit] = label_fruit

        cv2.imwrite(f'dataset/images/image_{i}.png', img)
        cv2.imwrite(f'dataset/labels/image_{i}_label.png', mask)
        i += 1